In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os
abs_dir = os.getcwd()
rel_dir = os.path.join(abs_dir, '../../data/raw/hospital/AZmariaMiddelares.xlsx')
df = pd.read_excel(rel_dir, sheet_name="Sheet1")

### Reformat dataset and write to excel

In [123]:
patients_dict={}
id_group=df.groupby(['Opnamenummer'])
for g_idx, group in id_group:
    # filter out 'ZZ-Niet ingevuld' in column 'Bestemming'
    if group['Bestemming na ontslag'].values[0] != 'ZZ-Niet ingevuld':
        # add patient ID
        patients_dict.update({g_idx: {}})
        # add patient age
        patients_dict[g_idx].update({'Age':group['Leeftijd'].values[0]})
        # add patient sex
        if group['Geslacht'].values[0] == 'Man':
            patients_dict[g_idx].update({'Sex': 'M'})
        else:
            patients_dict[g_idx].update({'Sex': 'F'})
        # add patient recovered or dead
        if group['Bestemming na ontslag'].values[0] != 'Overleden':
            patients_dict[g_idx].update({'Status': 'R'})
        else:
            patients_dict[g_idx].update({'Status': 'D'})
        # add patients wards as list, neglect emergency room and buffer
        wards=[]
        lengths=[]
        for r_idx, row in group.iterrows(): 
            if (group['Afdeling'][r_idx] != 'Spoed' and group['Afdeling'][r_idx] != 'Bufferafdeling'):
                wards.append(group['Afdeling'][r_idx])
                lengths.append(group['Ligduur'][r_idx])
        patients_dict[g_idx].update({'Wards': wards[::-1]})
        patients_dict[g_idx].update({'Lengths': lengths[::-1]})

In [217]:
age_R=[]
sex_R=[]
dICU_R=[]
dICUrec_R=[]

age_D=[]
sex_D=[]
dICU_D=[]
dICUrec_D=[]

age_C_R=[]
sex_C_R=[]
dC_R=[]

age_C_D=[]
sex_C_D=[]
dC_D=[]

age_ER=[]
sex_ER=[]
ER_status=[]

age_merged=[]
sex_merged=[]
icu_merged=[]

for ID in patients_dict:
    # if any entry in wards is ICU,
    if any(patients_dict[ID]['Wards'][idx] == 'IZ' for idx,ward in enumerate(patients_dict[ID]['Wards'])):
        age_merged.append(patients_dict[ID]['Age'])
        sex_merged.append(patients_dict[ID]['Sex'])
        icu_merged.append(1)
        # loop over wards and indices
        for idx,ward in enumerate(patients_dict[ID]['Wards']):
            # get index of ICU in wards
            if ward == 'IZ':
                # if recovered
                if patients_dict[ID]['Status'] == 'R':
                    # check if patient had a recovery stay in cohort or not and extract total length
                    if len(patients_dict[ID]['Wards'])-1 > idx:
                        dICUrec = sum(patients_dict[ID]['Lengths'][idx+1:])
                    else:
                        dICUrec=0
                    # add patient data to lists
                    age_R.append(patients_dict[ID]['Age'])
                    sex_R.append(patients_dict[ID]['Sex'])
                    dICU_R.append(patients_dict[ID]['Lengths'][idx])
                    dICUrec_R.append(dICUrec)
                # if dead
                else:
                    # check if patient had a recovery stay in cohort or not and extract total length
                    if len(patients_dict[ID]['Wards'])-1 > idx:
                         dICUrec = sum(patients_dict[ID]['Lengths'][idx+1:])
                    else:
                        dICUrec=0
                    # add patient data to lists
                    age_D.append(patients_dict[ID]['Age'])
                    sex_D.append(patients_dict[ID]['Sex'])
                    dICU_D.append(patients_dict[ID]['Lengths'][idx])
                    dICUrec_D.append(dICUrec)
    # if the patient didn't stay in ICU but did stay in Cohort
    elif any( (patients_dict[ID]['Wards'][idx] == 'Cohort-afdeling D501') or (patients_dict[ID]['Wards'][idx] == 'Cohort-afdeling D601') for idx,ward in enumerate(patients_dict[ID]['Wards'])):
        age_merged.append(patients_dict[ID]['Age'])
        sex_merged.append(patients_dict[ID]['Sex'])
        icu_merged.append(0)
        if patients_dict[ID]['Status'] == 'R':
            # sum length over all wards
            dC_R.append(sum(patients_dict[ID]['Lengths']))
            age_C_R.append(patients_dict[ID]['Age'])
            sex_C_R.append(patients_dict[ID]['Sex'])
        else:
            dC_D.append(sum(patients_dict[ID]['Lengths']))
            age_C_D.append(patients_dict[ID]['Age'])
            sex_C_D.append(patients_dict[ID]['Sex'])
    else:
        age_ER.append(patients_dict[ID]['Age'])
        sex_ER.append(patients_dict[ID]['Sex'])
        ER_status.append(patients_dict[ID]['Status'])

        
ICU_R_dict={
    'age': age_R,
    'sex': sex_R,
    'dICU': dICU_R,
    'dC_{ICU,rec}': dICUrec_R
}

ICU_D_dict={
    'age': age_D,
    'sex': sex_D,
    'dICU': dICU_D,
    'dC_{ICU,rec}': dICUrec_D
}

C_R_dict={
    'age': age_C_R,
    'sex': sex_C_R,
    'dC': dC_R,
}


C_D_dict={
    'age': age_C_D,
    'sex': sex_C_D,
    'dC': dC_D,
}

ER_dict={
    'age': age_ER,
    'sex': sex_ER,
    'recovered': ER_status
}

merged_dict={
    'age': age_merged,
    'sex': sex_merged,
    'icu': icu_merged
}

df_ICU_R = pd.DataFrame(ICU_R_dict)
df_ICU_D = pd.DataFrame(ICU_D_dict)
df_C_R = pd.DataFrame(C_R_dict)
df_C_D = pd.DataFrame(C_D_dict)
df_ER = pd.DataFrame(ER_dict)
df_merged = pd.DataFrame(merged_dict)

In [207]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('twallema_AZMM.xlsx')

# Write each dataframe to a different worksheet.
df_ICU_R.to_excel(writer, sheet_name='ICU (recovered)',index=False)
df_ICU_D.to_excel(writer, sheet_name='ICU (dead)',index=False)
df_C_R.to_excel(writer, sheet_name='Cohort (recovered)',index=False)
df_C_D.to_excel(writer, sheet_name='Cohort (dead)',index=False)
df_ER.to_excel(writer, sheet_name='Emergency room',index=False)
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [218]:
df_merged.head()
bins = pd.cut(df_merged['age'], [0, 10, 20, 30, 40, 50, 60, 70, 80,120])
counts = df_merged.groupby(bins)['age'].agg(['count'])
print(counts)

           count
age             
(0, 10]        0
(10, 20]       1
(20, 30]       0
(30, 40]       2
(40, 50]      14
(50, 60]      14
(60, 70]      26
(70, 80]      47
(80, 120]     72


In [219]:
df_merged.head()
bins = pd.cut(df_merged['age'], [0, 10, 20, 30, 40, 50, 60, 70, 80,120])
icu = df_merged.groupby(bins)['icu'].sum()
print(icu)

age
(0, 10]       0
(10, 20]      0
(20, 30]      0
(30, 40]      0
(40, 50]      5
(50, 60]      2
(60, 70]     14
(70, 80]     18
(80, 120]     9
Name: icu, dtype: int64


In [194]:
bins = pd.cut(df_ICU_R['age'], [0, 10, 20, 30, 40, 50, 60, 70, 80,120])
counts = df_ICU_R.groupby(bins)['age','dICU'].mean()
print(counts)

                 age       dICU
age                            
(0, 10]          NaN        NaN
(10, 20]         NaN        NaN
(20, 30]         NaN        NaN
(30, 40]         NaN        NaN
(40, 50]   46.666667  17.021526
(50, 60]   57.000000   6.015276
(60, 70]   66.181818   8.854292
(70, 80]   74.750000   7.836023
(80, 120]  82.000000  13.019084


<ipython-input-194-724a02e69f0d>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  counts = df_ICU_R.groupby(bins)['age','dICU'].mean()
